### A. Librerias

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Función para obtener datetime a partir de un unix timestamp y si es NaN devuelve NaN
def unixtime_to_datetime(unixtime):
    try:
        unixtime = unixtime/1000
        return datetime.datetime.fromtimestamp(int(unixtime)).strftime('%Y-%m-%d')
    except:
        return ''

### B. Leer datos

#### Lista de investigadores

In [3]:
# 1. Leer lista de investigadores
df_investigadores = pd.read_parquet('processed/parquet/01_lista_investigadores_renacyt.parquet')
df_investigadores = df_investigadores.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_investigadores = df_investigadores[['ctiVitae','orcid','institucionLaboralActual', 'nombres', 'apellidoPaterno', 'apellidoMaterno']]

df_investigadores['nombreCompleto'] = df_investigadores['nombres'] + ' ' + df_investigadores['apellidoPaterno'] + ' ' + df_investigadores['apellidoMaterno']
df_investigadores.drop(columns=['nombres','apellidoPaterno','apellidoMaterno'], inplace=True)
df_investigadores.drop_duplicates(inplace=True)
df_investigadores.reset_index(drop=True,inplace=True)

df_investigadores['ctiVitae'] = df_investigadores['ctiVitae'].astype(str)
df_investigadores['urlRenacyt'] = 'https://ctivitae.concytec.gob.pe/buscador-ui/#/ficha/ficha-renacyt?idInvestigador=' + df_investigadores['ctiVitae']
df_investigadores['urlOrcid'] = 'https://orcid.org/' + df_investigadores['orcid']

# 2. Guardar
df_investigadores.to_parquet('processed/resumen_parquet/a_investigadores_renacyt.parquet', index=False)
df_investigadores.to_excel('processed/resumen_excel/a_investigadores_renacyt.xlsx', index=False)
df_investigadores.tail(3)

,ctiVitae,orcid,institucionLaboralActual,nombreCompleto,urlRenacyt,urlOrcid
7208,356746,0000-0001-5650-3668,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE UFRN,ANDRES ORTIZ SALAZAR,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0001-5650-3668
7209,358819,0000-0001-7768-9932,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO (UFRJ),ALEX XIMENES NAVES,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0001-7768-9932
7210,363589,0009-0006-7870-6857,None,JOSELINE RODRIGUEZ PUMA,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0009-0006-7870-6857


#### Publicaciones extraidas de renacyt

In [4]:
# 1. Leer y procesar las publicaciones que están en renacyt
df_publicaciones_renacyt = pd.read_parquet('processed/parquet/03_publicaciones_renacyt.parquet')
df_publicaciones_renacyt = df_publicaciones_renacyt.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_publicaciones_renacyt = df_publicaciones_renacyt[['cti_vitae','quartil','descTipoProduccionBibliografica','titulo','revista','idDoi','fechaProduccion']]

df_publicaciones_renacyt.rename(columns={'cti_vitae':'ctiVitae', 'idDoi':'urlPublicacion'},inplace=True)
df_publicaciones_renacyt.drop_duplicates(inplace=True)
df_publicaciones_renacyt.reset_index(drop=True,inplace=True)
df_publicaciones_renacyt = df_publicaciones_renacyt[df_publicaciones_renacyt['fechaProduccion']>0]

df_publicaciones_renacyt['fechaProduccion'] = df_publicaciones_renacyt['fechaProduccion'].apply(unixtime_to_datetime)
df_publicaciones_renacyt['fechaProduccion'] = pd.to_datetime(df_publicaciones_renacyt['fechaProduccion'])
df_publicaciones_renacyt['añoProduccion'] = df_publicaciones_renacyt['fechaProduccion'].dt.year

df_publicaciones_renacyt['quartil'] = df_publicaciones_renacyt['quartil'].apply(lambda x: x.upper() if type(x)==str else x)
df_publicaciones_renacyt['quartil'] = df_publicaciones_renacyt['quartil'].apply(lambda x: x.strip() if type(x)==str else x)
df_publicaciones_renacyt['quartil'] = df_publicaciones_renacyt['quartil'].apply(lambda x: x if x in ['Q1','Q2','Q3','Q4'] else 'Sin quartil')
df_publicaciones_renacyt['ctiVitae'] = df_publicaciones_renacyt['ctiVitae'].astype(str)
df_publicaciones_renacyt['fuenteExtraccionData'] = 'Renacyt'
print(df_publicaciones_renacyt.shape)

# 2. Unir los investigadores con sus publicaciones de renacyt
df_investigadores_aux = df_investigadores.drop_duplicates(subset=['ctiVitae'],keep='first')
df_publicaciones_renacyt = df_investigadores_aux.merge(df_publicaciones_renacyt, on='ctiVitae', how='inner')

# 3. Ordenar y guardar
df_publicaciones_renacyt = df_publicaciones_renacyt[['ctiVitae', 'orcid', 'institucionLaboralActual', 'nombreCompleto', 'fuenteExtraccionData',
                                                      'descTipoProduccionBibliografica','titulo', 'revista', 'quartil', 'fechaProduccion', 
                                                      'añoProduccion', 'urlRenacyt', 'urlOrcid','urlPublicacion']]

df_publicaciones_renacyt.to_parquet('processed/resumen_parquet/a_publicaciones_renacyt.parquet', index=False)
df_publicaciones_renacyt.to_excel('processed/resumen_excel/b_publicaciones_renacyt.xlsx', index=False)
print(df_publicaciones_renacyt.shape)
df_publicaciones_renacyt.head(2)

(71072, 9)
(71072, 14)


,ctiVitae,orcid,institucionLaboralActual,nombreCompleto,fuenteExtraccionData,descTipoProduccionBibliografica,titulo,revista,quartil,fechaProduccion,añoProduccion,urlRenacyt,urlOrcid,urlPublicacion
0,10,0000-0003-1583-7113,UNIVERSIDAD DE LIMA,SILVIA PATRICIA PONCE ALVAREZ,Renacyt,Conference Paper,Development and evaluation of an autonomous an...,Proceedings of SPIE - The International Societ...,Sin quartil,2023-01-01,2023,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0003-1583-7113,www.doi.org/10.1117/12.2647950
1,10,0000-0003-1583-7113,UNIVERSIDAD DE LIMA,SILVIA PATRICIA PONCE ALVAREZ,Renacyt,Art\xedculo en revista cient\xedfica,The Applicability of Cellulose \u2014 Tara Gum...,"Water, Air, and Soil Pollution",Q2,2022-08-01,2022,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0003-1583-7113,www.doi.org/10.1007/S11270-022-05818-Z


#### Publicaciones extraidas de orcid

In [5]:
# 1. Leer y procesar las publicaciones que están en orcid
df_publicaciones_orcid = pd.read_parquet('processed/parquet/06_trabajos_orcid.parquet')
df_publicaciones_orcid = df_publicaciones_orcid.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df_publicaciones_orcid.drop_duplicates(inplace=True)
df_publicaciones_orcid.reset_index(drop=True,inplace=True)

df_publicaciones_orcid['publication_year'] = df_publicaciones_orcid['publication_year'].fillna(0)
df_publicaciones_orcid['publication_year'] = df_publicaciones_orcid['publication_year'].astype(int)
df_publicaciones_orcid = df_publicaciones_orcid[df_publicaciones_orcid['publication_year']>0]

df_publicaciones_orcid['fechaProduccion'] = df_publicaciones_orcid['publication_year'].astype(str) + '-' + df_publicaciones_orcid['publication_month'].astype(str) + '-' + df_publicaciones_orcid['publication_day'].astype(str)
df_publicaciones_orcid['orcid'] = df_publicaciones_orcid['orcid'].astype(str)
df_publicaciones_orcid['fuenteExtraccionData'] = 'orcid'
print(df_publicaciones_orcid.shape)

# 2. Unir los investigadores con sus publicaciones de orcid
df_investigadores_aux = df_investigadores.drop_duplicates(subset=['orcid'],keep='first')
df_publicaciones_orcid = df_investigadores_aux.merge(df_publicaciones_orcid, on='orcid', how='inner')
print(df_publicaciones_orcid.shape)

# 3. Renombrar, ordenar y guardar
df_publicaciones_orcid['quartil'] = ''
df_publicaciones_orcid.rename(columns={'doi_url':'urlPublicacion', 'title':'titulo', 'journal_title':'revista', 'publication_year':'añoProduccion', 'journal_title':'revista', 'work_type':'descTipoProduccionBibliografica'}, inplace=True)

df_publicaciones_orcid = df_publicaciones_orcid[['ctiVitae', 'orcid', 'institucionLaboralActual', 'nombreCompleto', 'fuenteExtraccionData',
                                                      'descTipoProduccionBibliografica','titulo', 'revista', 'quartil', 'fechaProduccion', 
                                                      'añoProduccion', 'urlRenacyt', 'urlOrcid','urlPublicacion']]

df_publicaciones_orcid.to_parquet('processed/resumen_parquet/b_publicaciones_orcid.parquet', index=False)
df_publicaciones_orcid.to_excel('processed/resumen_excel/c_publicaciones_orcid.xlsx', index=False)
df_publicaciones_orcid.head(2)

(94396, 15)
(94396, 20)


,ctiVitae,orcid,institucionLaboralActual,nombreCompleto,fuenteExtraccionData,descTipoProduccionBibliografica,titulo,revista,quartil,fechaProduccion,añoProduccion,urlRenacyt,urlOrcid,urlPublicacion
0,10,0000-0003-1583-7113,UNIVERSIDAD DE LIMA,SILVIA PATRICIA PONCE ALVAREZ,orcid,journal-article,Development and evaluation of an autonomous an...,International Workshop on Thin Films for Elect...,,2023-02-01,2023,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0003-1583-7113,https://doi.org/10.1117/12.2647950
1,10,0000-0003-1583-7113,UNIVERSIDAD DE LIMA,SILVIA PATRICIA PONCE ALVAREZ,orcid,journal-article,Treatment of irrigation water infested with ne...,DESALINATION AND WATER TREATMENT,,2020-None-None,2020,https://ctivitae.concytec.gob.pe/buscador-ui/#...,https://orcid.org/0000-0003-1583-7113,https://doi.org/10.5004/dwt.2020.24842
